In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
df_fashion = pd.read_csv('generated_reviews_fashion.csv')

In [3]:
len(df_fashion.index)

46

In [4]:
df_fashion.head()

,input_text,target_text,generated_text
0,cushioning | husband | insert | bone | plantar...,Pinnacle seems to have more cushioning so my h...,I have been using cushioning for the last 10 y...
1,price | product | size | chart | order | buy |...,"great price for the product, though the sizes ...",I was excited to find this product at such a c...
2,fit | wash | water | shrink | inch | inseam | ...,Good fit even after washing in hot water to fo...,I have a very nice Fit Wash that I can order a...
3,shoe | comfort | performance | quality | time ...,Great shoe. I've had Nike's before and have al...,I have been using these for over 30 years and ...
4,bit | picture | light | way | box | paper | fi...,It looks a bit nicer on the picture. Its very ...,I like this bit of a picture of the light way ...


In [5]:
df_software = pd.read_csv('generated_reviews_software.csv', names=['input_text','target_text','generated_text'])

In [6]:
len(df_software.index)

880

In [7]:
df_software.head()

,input_text,target_text,generated_text
0,version | software | pay | middle | update | f...,I just recently converted to this version from...,I have been using this version because I didn'...
1,tech | support | fact | method | product | res...,If you have any problems you will not be able ...,I really love using these products. I didn't r...
2,look | hood | information | engine | user | in...,"Because, while I'm not about to go looking und...",I was looking for a product that will help min...
3,desktop | sound | video | editing | software |...,Corel VideoStudio Ultimate X8 installed on my ...,I have used Desktop for years and loved it.......
4,experience | accounting | product | version | ...,My last experience with a Peachtree accounting...,I have had experience with accounting products...


In [8]:
df_appliances = pd.read_csv('generated_reviews_appliances.csv', names=['input_text','target_text','generated_text'])

In [9]:
len(df_appliances.index)

52

In [14]:
df_grocery = pd.read_csv('generated_reviews_appliances.csv')
df_grocery.head()

,input_text,target_text,generated_text
0,kit | clean | dryer | thing | deal | time,This is a complete kit for cleaning the dryer ...,I bought this kit to use as the clean out kit....
1,problem | lint | house | vent | dryer | filter...,Anyone who thinks they don't have a problem wi...,I have a problem with lint in the house vents....
2,today | go | roof | lint | drill | reverse | b...,"I used this today, going in from the roof.\n\n...",I think it is possible to use this after I sho...
3,equipment | rubber | experience | peace | mind...,I used these (2) hoses to replace the original...,I have a pneumatizer with rubber in it and I h...
4,get | burner | value | dealer,Getting these burner covers at stores used to ...,Get Started using this burner! I didn't realiz...


In [15]:
len(df_grocery.index)

51

In [18]:
df = df_fashion.append(df_appliances, ignore_index=True)

In [19]:
df = df.append(df_software, ignore_index=True)
df = df.append(df_grocery, ignore_index=True )

In [20]:
df.head()

,input_text,target_text,generated_text
0,cushioning | husband | insert | bone | plantar...,Pinnacle seems to have more cushioning so my h...,I have been using cushioning for the last 10 y...
1,price | product | size | chart | order | buy |...,"great price for the product, though the sizes ...",I was excited to find this product at such a c...
2,fit | wash | water | shrink | inch | inseam | ...,Good fit even after washing in hot water to fo...,I have a very nice Fit Wash that I can order a...
3,shoe | comfort | performance | quality | time ...,Great shoe. I've had Nike's before and have al...,I have been using these for over 30 years and ...
4,bit | picture | light | way | box | paper | fi...,It looks a bit nicer on the picture. Its very ...,I like this bit of a picture of the light way ...


In [21]:
len(df.index)

1029

In [22]:
import nltk
nltk.download('stopwords')
import re
import string
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [23]:
def clean_text(tweet:str) -> str:
    """
    Convert all text to lowercase, remove stock market tickers, RT symbol, hyperlinks and the hastag symbol
    :param tweet: tweet by a unique user
    :return: cleaned string without hashtags, emojis, and punctuation
    """
    # make text lower case
    tweet = tweet.lower()
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', str(tweet))
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', str(tweet))
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', str(tweet))
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', str(tweet))
    
    # remove punctuation
    punct = set(string.punctuation)
    tweet = "".join(ch for ch in tweet if ch not in punct)
    
    # remove stopwords
    stop_words = set(stopwords.words("english"))
    tweet = " ".join(word for word in tweet.split() if word not in stop_words)
    
    return tweet

In [24]:
df["generated_text"] = df["generated_text"].apply(clean_text)

In [25]:
df["target_text"] = df["target_text"].apply(clean_text)

In [26]:
# Roberta model
df_bert = pd.DataFrame()
column_names = ["titletext", "label"]


In [27]:
df_bert.head()

""


In [28]:
df.head()

,input_text,target_text,generated_text
0,cushioning | husband | insert | bone | plantar...,pinnacle seems cushioning husband likes better...,using cushioning last 10 years first introduce...
1,price | product | size | chart | order | buy |...,great price product though sizes tend bigger b...,excited find product cheap price couldnt live ...
2,fit | wash | water | shrink | inch | inseam | ...,good fit even washing hot water force shrinkin...,nice fit wash order time small inseam much big...
3,shoe | comfort | performance | quality | time ...,great shoe ive nikes always pleased comfort pe...,using 30 years delighted surprised tried found...
4,bit | picture | light | way | box | paper | fi...,looks bit nicer picture light way held box won...,like bit picture light way pictures show simpl...


In [29]:
df['input_text'][1]

'price | product | size | chart | order | buy | difference'

In [30]:
for i in range (len(df['target_text'])):
  j = len(df_bert.index)
  df_bert.loc[j, 'titletext'] = df['target_text'][i]
  df_bert.loc[j, 'label'] = int(0)

In [31]:
for i in range (len(df['generated_text'])):
  j = len(df_bert.index)
  df_bert.loc[j, 'titletext'] = df['generated_text'][i]
  df_bert.loc[j, 'label'] = int(1)

In [32]:
df_bert['label'] = df_bert['label'].astype('int')

In [33]:
df_bert.head()

,titletext,label
0,pinnacle seems cushioning husband likes better...,0
1,great price product though sizes tend bigger b...,0
2,good fit even washing hot water force shrinkin...,0
3,great shoe ive nikes always pleased comfort pe...,0
4,looks bit nicer picture light way held box won...,0


In [26]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_bert, test_size=0.2)

In [34]:
df_bert['label'].value_counts()

0    1029
1    1029
Name: label, dtype: int64

In [37]:
df_bert.to_csv("reviews.csv", index=False)

In [ ]:
# !pip install transformers

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.layers import Dense, Input
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.models import Model
# from tensorflow.keras.callbacks import ModelCheckpoint
# import numpy as np

# import transformers
# from tqdm.notebook import tqdm
# from tokenizers import BertWordPieceTokenizer

In [ ]:
# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# def bert_encode(input_text, max_len):
#     input_ids = []
#     attension_masks = []
#     for text in input_text:
#         output_dict = tokenizer.encode_plus(
#             text, 
#             add_special_tokens = True,
#             truncation=True,
#             max_length = max_len,
#             pad_to_max_length = True,
#             return_attention_mask = True
#         )
#         input_ids.append(output_dict['input_ids'])
#         attension_masks.append(output_dict['attention_mask'])
#     return np.array(input_ids), np.array(attension_masks)

In [ ]:
# text = df_bert['titletext']
# target = df_bert['label']
# train_input_ids, train_attention_masks = bert_encode(text, 60)

In [ ]:
# def create_model(bert_model):
#     input_ids = tf.keras.Input(shape= (60,), dtype= 'int32')
#     attention_masks = tf.keras.Input(shape= (60,), dtype= 'int32')
    
#     output = bert_model([input_ids, attention_masks])
#     output = output[1]
#     output = tf.keras.layers.Dense(32, activation= 'relu')(output)
#     output = tf.keras.layers.Dropout(0.2)(output)
#     output = tf.keras.layers.Dense(1, activation= 'sigmoid')(output)
    
#     model = tf.keras.models.Model(inputs= [input_ids, attention_masks], outputs= output)
#     model.compile(Adam(learning_rate=1e-5), loss= 'binary_crossentropy', metrics= ['accuracy'])
#     return model

In [ ]:
# from transformers import TFBertModel
# bert_model = TFBertModel.from_pretrained('bert-base-uncased')

In [ ]:
# model = create_model(bert_model)
# model.summary()

In [ ]:
%%time

history = model.fit(
    [train_input_ids, train_attention_masks],
    target, 
    validation_split = 0.2,
    epochs = 3,
    batch_size = 10
)

In [ ]:
# check_input_ids, check_attention_masks = bert_encode("Im checking if its working", 60)

In [ ]:
# model.predict([check_input_ids]).argmax(axis=-1)
